## Robot@Home 2 - Scenes



`R@H2 notebook series. Version 1.0`



### Scenes



Scenes contain the 3D coloured point cloud reconstruction for each room. Such
reconstruction was built employing a mapping application which registers the
RGB-D observations employing different ICP-based techniques. The 3D point clouds
are stored as plain text files containing the coordinates and colors of the
points that make up the 3D cloud. These files can be easily visualized with
current software for the visualization of point clouds like [MeshLab](https://www.meshlab.net/).

![img](./70-scenes-meshlab.png "12-scene.txt displayed thorugh MeshLab")



In [1]:
import os
import pandas as pd
from robotathome import RobotAtHome
from robotathome import logger, log, set_log_level
from robotathome import time_win2unixepoch, time_unixepoch2win

In [2]:
log.set_log_level('INFO')  # SUCCESS is the default

my_rh_path = '~/WORKSPACE/R@H2-2.0.1'
my_rgbd_path = os.path.join(my_rh_path, 'files/rgbd')
my_scene_path = os.path.join(my_rh_path, 'files/scene')
my_wspc_path = '~/WORKSPACE'

try:
    rh = RobotAtHome(my_rh_path, my_rgbd_path, my_scene_path, my_wspc_path)
except:
    logger.error("Something was wrong")

2022-01-22 22:49:49.846 | SUCCESS  | robotathome.core.reader:__open_dataset:87 - Connection is established: rh.db


As usual, this toolbox provides a function to get scenes



In [3]:
scenes = rh.get_scenes()
logger.info("\nScenes: \n{}", scenes)

2022-01-22 22:49:49.862 | INFO     | __main__:<module>:2 - 
Scenes: 
    id  home_session_id  home_subsession_id  home_id  room_id  \
0    0                0                   0        0        0   
1    1                0                   0        0        1   
2    2                0                   0        0        3   
3    3                0                   0        0        4   
4    4                0                   0        0        5   
..  ..              ...                 ...      ...      ...   
67  67                6                   1        4       36   
68  68                6                   0        4       39   
69  69                6                   1        4       39   
70  70                6                   0        4       40   
71  71                6                   1        4       40   

                                           scene_file  
0   /home/goyo/WORKSPACE/R@H2-2.0.1/files/scene/se...  
1   /home/goyo/WORKSPACE/R@H2-2.0.1/fi

After some notebooks we have enough skills to advance with an example of multi
column filtering



In [4]:
# Let's suppose we want to get the scene file of the kitchen of the home
# session alma-s1

# Get the ids from their names
hs_id = rh.name2id('alma-s1','hs')
r_id = rh.name2id('alma_kitchen1','r')

# Let's query the scene dataframe with the ids
scene =  scenes.query(f'home_session_id=={hs_id} & room_id=={r_id}')
# Extract the value of the scene_file field
scene_file = scene.scene_file.values[0]
logger.info(f'\nScene file: \n{scene_file}')

2022-01-22 22:49:49.886 | INFO     | __main__:<module>:12 - 
Scene file: 
/home/goyo/WORKSPACE/R@H2-2.0.1/files/scene/session_1/alma/kitchen1/subsession_1/2_scene.txt


### Labeled 3D point maps



Each reconstructed room, i.e. scene has been labeled setting bounding boxes to
the objects appearing in the point cloud reconstruction and including annotations
with the ground truth information about their category, e.g. counter, book,
couch, shelf, as well as an object id to identify the particular instance, i.e.
counter-1, book-3, etc.

To get the labels for a scene use the following function



In [5]:
# let's extract the labels for any scene, for example the scene with id = 0
id = scene['id'].values[0]
scene_labels = rh.get_scene_labels(id)
logger.info(f'\nScene labels: {scene_labels.info()}')

2022-01-22 22:49:49.906 | INFO     | __main__:<module>:4 - 
Scene labels: None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             32 non-null     int64  
 1   local_id       32 non-null     int64  
 2   scene_id       32 non-null     int64  
 3   object_id      32 non-null     int64  
 4   object_name    32 non-null     object 
 5   bb_pose_x      32 non-null     float64
 6   bb_pose_y      32 non-null     float64
 7   bb_pose_z      32 non-null     float64
 8   bb_pose_yaw    32 non-null     float64
 9   bb_pose_pitch  32 non-null     float64
 10  bb_pose_roll   32 non-null     float64
 11  bb_corner1_x   32 non-null     float64
 12  bb_corner1_y   32 non-null     float64
 13  bb_corner1_z   32 non-null     float64
 14  bb_corner2_x   32 non-null     float64
 15  bb_corner2_y   32 non-null     float64
 16  bb_corner2_z   32 non-null     float64
dtypes: float64(12), int64(4), object(1)
memory usage: 4.4+ K

As you can see, for each label there is data about its pose and 3D coordinates.



In [6]:
logger.info(f'\nThere are {len(scene_labels)} object labels in the scene {id}\n')
# Let's examine the first bounding box
local_id = 0
logger.info('\nData about the label {}', scene_labels.query('local_id==0').loc[0,:])

2022-01-22 22:49:49.912 | INFO     | __main__:<module>:1 - 
There are 32 object labels in the scene 2

2022-01-22 22:49:49.916 | INFO     | __main__:<module>:4 - 
Data about the label id                           40
local_id                      0
scene_id                      2
object_id                    42
object_name      wall_cabinet_0
bb_pose_x               1.30764
bb_pose_y              0.444265
bb_pose_z                 -0.08
bb_pose_yaw            0.920096
bb_pose_pitch         -0.012116
bb_pose_roll          -0.015913
bb_corner1_x            1.73182
bb_corner1_y           0.095152
bb_corner1_z          -0.011503
bb_corner2_x           0.876149
bb_corner2_y           0.833158
bb_corner2_z            1.73173
Name: 0, dtype: object


Each box delimits an object for which the dataset has additional information
about its features. If you consider this information of interest you can obtain
it by accessing the extended set of labels. Please, note that this extended set
is experimental: object info has been obtained from 2D observations and the
bounding box set has slightly more records (#2446) than the objects one (#2125).



In [7]:
# let's extract the labels for any scene, for example the scene with id = 0
id = scene['id'].values[0]
scene_labels = rh.get_scene_labels(id, True)
logger.info(f'\nScene labels: {scene_labels.info()}')

2022-01-22 22:49:49.938 | INFO     | __main__:<module>:4 - 
Scene labels: None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      32 non-null     int64  
 1   local_id                32 non-null     int64  
 2   scene_id                32 non-null     int64  
 3   object_id               32 non-null     int64  
 4   object_name             32 non-null     object 
 5   object_type_id          32 non-null     int64  
 6   bb_pose_x               32 non-null     float64
 7   bb_pose_y               32 non-null     float64
 8   bb_pose_z               32 non-null     float64
 9   bb_pose_yaw             32 non-null     float64
 10  bb_pose_pitch           32 non-null     float64
 11  bb_pose_roll            32 non-null     float64
 12  bb_corner1_x            32 non-null     float64
 13  bb_corner1_y            32 non-null     float64
 14  bb_corner1_z            32 non-null     floa

Continuing the example



In [8]:
logger.info(f'\nThere are {len(scene_labels)} object labels in the scene {id}\n')
# Let's examine again the first bounding box
logger.info('\nData about the label \n{}', scene_labels.query('local_id==0').loc[0,:])

2022-01-22 22:49:49.946 | INFO     | __main__:<module>:1 - 
There are 32 object labels in the scene 2

2022-01-22 22:49:49.952 | INFO     | __main__:<module>:3 - 
Data about the label 
id                                    40
local_id                               0
scene_id                               2
object_id                             42
object_name               wall_cabinet_0
object_type_id                        24
bb_pose_x                        1.30764
bb_pose_y                       0.444265
bb_pose_z                          -0.08
bb_pose_yaw                     0.920096
bb_pose_pitch                  -0.012116
bb_pose_roll                   -0.015913
bb_corner1_x                     1.73182
bb_corner1_y                    0.095152
bb_corner1_z                   -0.011503
bb_corner2_x                    0.876149
bb_corner2_y                    0.833158
bb_corner2_z                     1.73173
planarity                       0.441967
scatter                         0.55